In [1]:
import requests
import pandas as pd

url = "http://192.168.31.130:32327/games_reviews/_search?scroll=1m"  # 使用 Scroll，scroll=1m 表示 Scroll 保持活躍1分鐘
headers = {"Content-Type": "application/json"}

# 初始化 Scroll 查詢，不指定欄位，這樣會返回所有欄位
data = {
    "_source": True,  # 設置為 True 表示返回所有欄位
    "query": {"match_all": {}},  # 匹配所有文件
    "size": 1000,  # 每次返回1000筆資料
}

response = requests.get(url, headers=headers, json=data)
response_json = response.json()

# 提取初始查詢的 hits 和 scroll_id
hits = response_json["hits"]["hits"]
scroll_id = response_json["_scroll_id"]

# 存儲資料的列表
data_list = [hit["_source"] for hit in hits]

# 使用 Scroll API 繼續獲取剩餘的資料
while len(hits) > 0:
    scroll_url = "http://192.168.31.130:32327/_search/scroll"
    scroll_data = {
        "scroll": "1m",  # Scroll 保持活躍1分鐘
        "scroll_id": scroll_id,  # 使用前次查詢返回的 scroll_id 繼續查詢
    }

    scroll_response = requests.get(scroll_url, headers=headers, json=scroll_data)
    scroll_response_json = scroll_response.json()

    hits = scroll_response_json["hits"]["hits"]
    scroll_id = scroll_response_json["_scroll_id"]

    # 將新獲取的 hits 資料追加到列表中
    data_list.extend([hit["_source"] for hit in hits])

# 將提取的資料轉換為 DataFrame
df = pd.DataFrame(data_list)

# 顯示 DataFrame
df


,earlyAccess,itemType,reviewScore,developers,itemCode,unreleased,owners,playtimeData,steamId,firstReleaseDate,...,tags,followers,estimateDetails,name,Online_by_month,wishlists,dlc,totalRevenue,EAReleaseDate,earlyAccessExitDate
0,False,game,96,[Valve],0,False,22270700,"{'median': 17.066666666666666, 'distribution':...",240,1099285200000,...,"[Shooter, FPS, Action, Multiplayer, Team-Based...",439910.0,"{'reviewBased': 7178013.503769911, 'rankBased'...",Counter-Strike: Source,"[[Last 30, 8436.63, -1847.2, -17.96%, 13793], ...",383900.0,NaN,NaN,NaN,NaN
1,False,game,91,[Valve],0,False,15470319,"{'median': 1.2, 'distribution': {'500-1000h': ...",80,1078117200000,...,"[Action, FPS, Shooter, Multiplayer, First-Pers...",46784.0,"{'reviewBased': 1261131.3876663663, 'rankBased...",Counter-Strike: Condition Zero,"[[Last 30, 306.69, -2.4, -0.78%, 537], [August...",67100.0,NaN,NaN,NaN,NaN
2,False,game,82,[Realmforge Studios],0,False,2032685,"{'median': 3.4166666666666665, 'distribution':...",262280,1429848000000,...,"[Strategy, RPG, Management, Fantasy, Dungeon C...",32067.0,"{'playtimeBased': 866631, 'reviewBased': 38326...",Dungeons 2,"[[Last 30, 18.74, -1.5, -7.29%, 52], [August 2...",62700.0,"[{'steamId': 408600, 'revenue': 0, 'releaseDat...",4105373.0,NaN,NaN
3,False,game,60,[Code Club AB],0,False,276068,"{'median': 0.5, 'distribution': {'500-1000h': ...",1179680,1595563200000,...,"[Free to Play, Sandbox, Massively Multiplayer,...",7384.0,"{'playtimeBased': 256259, 'reviewBased': 20172...",Wurm Online,"[[Last 30, 278.63, +115.6, +70.91%, 506], [Aug...",17600.0,NaN,NaN,NaN,NaN
4,False,game,84,[Hendrik Felix Pohl],0,False,316060,"{'median': 3.3333333333333335, 'distribution':...",688130,1551330000000,...,"[Psychological Horror, Precision Platformer, D...",4506.0,"{'playtimeBased': 163200, 'reviewBased': 25420...",Pogostuck: Rage With Your Friends,"[[Last 30, 48.54, -6.7, -12.14%, 100], [August...",45100.0,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10331,True,game,79,[Chimera Entertainment],0,False,12099,NaN,2195410,1717581314557,...,"[Early Access, Atmospheric, Singleplayer, Mult...",17415.0,"{'playtimeBased': 10281, 'reviewBased': 19182,...",Songs of Silence,"[[Last 30, 22.17, +5.5, +32.91%, 82], [August ...",165000.0,"[{'steamId': 2685650, 'revenue': 3678, 'releas...",398917.0,1.717581e+12,NaN
10332,True,game,70,[Bloodbath Studios],0,False,9172,NaN,2293680,1721266026708,...,"[Survival, Crafting, Adventure, Open World Sur...",4546.0,"{'playtimeBased': 6566, 'reviewBased': 16848, ...",Crestfallen: Medieval Survival,"[[Last 30, 20.69, -6.3, -23.45%, 61], [August ...",31900.0,"[{'steamId': 3114190, 'revenue': 10, 'releaseD...",103874.0,1.721266e+12,NaN
10333,False,game,54,[Transhuman Design],0,False,42137,NaN,262100,1460001600000,...,"[Action, Casual, Indie, Pixel Graphics, Destru...",534.0,"{'reviewBased': 5610.708333333334, 'rankBased'...",Trench Run,"[[Last 30, 0.00, -0.0, -100.00%, 0], [Septembe...",2200.0,NaN,NaN,NaN,NaN
10334,False,game,92,[VV-LABO],0,False,648,NaN,1277870,1657684800000,...,"[Competitive, Action, Casual, Physics, Colorfu...",624.0,"{'reviewBased': 648, 'rankBased': None}",SUPER UFO FIGHTER,[],17600.0,"[{'steamId': 2084910, 'revenue': 8, 'releaseDa...",3409.0,NaN,NaN


In [2]:
df.to_json('raw.json')

In [3]:
from elasticsearch import Elasticsearch
import pandas as pd
import requests

In [4]:
# 連接到 Elasticsearch
es = Elasticsearch(
    "http://192.168.31.130:32001"
)  # 將 "localhost:9200" 替換為你的 Elasticsearch 地址


# 初始化 scroll 查詢
scroll_size = 1000  # 每次抓取 1000 條數據
scroll_time = "2m"  # 設置 scroll 時間窗口為 2 分鐘


# 查詢 Elasticsearch，獲取資料
res = es.search(
    index="games_reviews",  # 替換成你想要查詢的索引名稱
    scroll=scroll_time,
    size=scroll_size,
    body={
        "_source": [
            "steamId",
            "name",
            "price",
            "reviews",
            "reviewsSteam",
            "tags",
            "genres",
            "features",
            "languages",
            "developers",
            "followers",
            "avgPlaytime",
            "reviewScore",
            "releaseDate",
            "earlyAccess",
            "wishlists",
            "copiesSold",
            "revenue",
            "players",
            "owners",
            "steamPercent",
            "itemType",
            "totalRevenue",
            "dlc",
            "history",
            "twitch_data",
            "Online_by_month",
        ],  # 你想要的欄位
    },
)

# 取得第一批數據並初始化 scroll_id
scroll_id = res["_scroll_id"]
all_hits = res["hits"]["hits"]  # 保存第一批數據

# 持續抓取後續的數據
while len(res["hits"]["hits"]) > 0:
    res = es.scroll(scroll_id=scroll_id, scroll=scroll_time)
    all_hits.extend(res["hits"]["hits"])  # 將每次抓取的數據加入到 all_hits 中

# 將結果轉換為 DataFrame
data = [doc["_source"] for doc in all_hits]
df = pd.DataFrame(data)

# 打印或保存 DataFrame
df = df[
    [
        "steamId",
        "name",
        "price",
        "reviews",
        "reviewsSteam",
        "tags",
        "genres",
        "features",
        "languages",
        "developers",
        "followers",
        "avgPlaytime",
        "reviewScore",
        "releaseDate",
        "earlyAccess",
        "wishlists",
        "copiesSold",
        "revenue",
        "players",
        "owners",
        "steamPercent",
        "itemType",
        "totalRevenue",
        "dlc",
        "history",
        "twitch_data",
        "Online_by_month",
    ]
]

df

C:\Users\student\AppData\Local\Temp\ipykernel_5820\2670690577.py:13: DeprecationWarning: Received 'size' via a specific parameter in the presence of a 'body' parameter, which is deprecated and will be removed in a future version. Instead, use only 'body' or only specific parameters.
  res = es.search(
C:\Users\student\AppData\Local\Temp\ipykernel_5820\2670690577.py:13: ElasticsearchWarning: Elasticsearch built-in security features are not enabled. Without authentication, your cluster could be accessible to anyone. See https://www.elastic.co/guide/en/elasticsearch/reference/7.17/security-minimal-setup.html to enable security.
  res = es.search(
C:\Users\student\AppData\Local\Temp\ipykernel_5820\2670690577.py:56: ElasticsearchWarning: Elasticsearch built-in security features are not enabled. Without authentication, your cluster could be accessible to anyone. See https://www.elastic.co/guide/en/elasticsearch/reference/7.17/security-minimal-setup.html to enable security.
  res = es.scroll(

,steamId,name,price,reviews,reviewsSteam,tags,genres,features,languages,developers,...,revenue,players,owners,steamPercent,itemType,totalRevenue,dlc,history,twitch_data,Online_by_month
0,240,Counter-Strike: Source,9.99,166086,120162.0,"[Shooter, FPS, Action, Multiplayer, Team-Based...",[Action],"[Cross-Platform Multiplayer, Steam Achievement...","[English, French, German, Italian, Japanese, K...",[Valve],...,9.045100e+07,14951135,22270700,0.723493,game,NaN,NaN,"[{'timeStamp': '2015-01-01 16:00:00', 'score':...","[Oct 2024, 6, -18, -75%, 48, 2, -, -, 7, 351, ...","[[Last 30, 8436.63, -1847.2, -17.96%, 13793], ..."
1,80,Counter-Strike: Condition Zero,9.99,24255,18859.0,"[Action, FPS, Shooter, Multiplayer, First-Pers...",[Action],"[Single-player, Valve Anti-Cheat enabled, Fami...","[English, French, German, Italian, Spanish - S...",[Valve],...,3.944117e+06,5209936,15470319,0.777530,game,NaN,NaN,"[{'timeStamp': '2015-01-01 16:00:00', 'score':...","[Oct 2024, 1, -1, -50%, 5, 1, -, -, 2, 14, Sep...","[[Last 30, 306.69, -2.4, -0.78%, 537], [August..."
2,262280,Dungeons 2,19.99,5015,2191.0,"[Strategy, RPG, Management, Fantasy, Dungeon C...","[RPG, Simulation, Strategy]","[Single-player, Cross-Platform Multiplayer, St...","[English, German, French, Italian, Spanish - S...",[Realmforge Studios],...,3.711132e+06,936282,2032685,0.436889,game,4105373.0,"[{'steamId': 408600, 'revenue': 0, 'releaseDat...","[{'timeStamp': '2015-04-24 20:00:00', 'score':...","[Sep 2024, 1, -, -, 8, 1, -, -, 2, 88, Aug 202...","[[Last 30, 18.74, -1.5, -7.29%, 52], [August 2..."
3,1179680,Wurm Online,0.00,2085,2085.0,"[Free to Play, Sandbox, Massively Multiplayer,...","[Adventure, Free To Play, Indie, Massively Mul...","[MMO, Online PvP, Online Co-op, Cross-Platform...",[English],[Code Club AB],...,1.607520e+06,147360,276068,1.000000,game,NaN,NaN,"[{'timeStamp': '2020-07-24 20:00:00', 'score':...","[Oct 2024, 17, -13, -43.3%, 74, 1, -, -, 3, 52...","[[Last 30, 278.63, +115.6, +70.91%, 506], [Aug..."
4,688130,Pogostuck: Rage With Your Friends,6.99,5649,5142.0,"[Psychological Horror, Precision Platformer, D...","[Action, Adventure, Casual, Indie]","[Single-player, Online PvP, Online Co-op, Stea...","[English, French, Italian, German, Spanish - S...",[Hendrik Felix Pohl],...,1.320316e+06,278016,316060,0.910250,game,NaN,NaN,"[{'timeStamp': '2019-02-28 21:00:00', 'score':...","[Oct 2024, 92, +6, +7%, 249, 5, +1, +25%, 16, ...","[[Last 30, 48.54, -6.7, -12.14%, 100], [August..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10331,2195410,Songs of Silence,39.99,417,417.0,"[Early Access, Atmospheric, Singleplayer, Mult...","[Indie, Strategy, Early Access]","[Single-player, Online PvP, Online Co-op, Stea...","[English, French, German, Spanish - Spain, Jap...",[Chimera Entertainment],...,3.911760e+05,12099,12099,1.000000,game,398917.0,"[{'steamId': 2685650, 'revenue': 3678, 'releas...","[{'timeStamp': '2024-06-06 01:55:14', 'score':...","[Oct 2024, 1, -36, -97.3%, 2, 1, -, -, 1, 3, S...","[[Last 30, 22.17, +5.5, +32.91%, 82], [August ..."
10332,2293680,Crestfallen: Medieval Survival,14.99,351,351.0,"[Survival, Crafting, Adventure, Open World Sur...","[Action, Adventure, Indie, Massively Multiplay...","[Single-player, MMO, Online PvP, Online Co-op,...",[English],[Bloodbath Studios],...,1.038640e+05,9172,9172,1.000000,game,103874.0,"[{'steamId': 3114190, 'revenue': 10, 'releaseD...","[{'timeStamp': '2024-07-18 17:27:06', 'score':...","[Oct 2024, 1, -61, -98.4%, 2, 1, -, -, 1, 1, S...","[[Last 30, 20.69, -6.3, -23.45%, 61], [August ..."
10333,262100,Trench Run,3.99,89,48.0,"[Action, Casual, Indie, Pixel Graphics, Destru...","[Action, Casual, Indie]","[Single-player, Online PvP, Shared/Split Scree...",[English],[Transhuman Design],...,1.210900e+04,6197,42137,0.539326,game,NaN,NaN,"[{'timeStamp': '2016-04-07 20:00:00', 'score':...","[Mar 2021, 1, -, -, 1, 1, -, -, 1, 1]","[[Last 30, 0.00, -0.0, -100.00%, 0], [Septembe..."
10334,1277870